In [ ]:
import os
import cv2
import json
import random
random.seed(77)

In [ ]:
json_dir_name = os.path.expanduser("~/Desktop/joo/data/korad/data/labeling/")
tiff_dir_name = os.path.expanduser("~/Desktop/joo/data/korad/data/images/")
save_dir = "./korad_boxes"

numbers = os.listdir(tiff_dir_name)
numbers.sort()

all_dict = []

In [ ]:
classes = ['AlligatorCrack', 'ConcreteCrack', 'Spalling', 'Efflorescene', 'Exposure']

colors = []
for i in range(len(classes)):
    rd1 = random.randint(0, 256)
    rd2 = random.randint(0, 256)
    rd3 = random.randint(0, 256)
    rd_color = (rd1, rd2, rd3)
    colors.append(rd_color)


labeling 이랑 image랑 file 명 맞춰서 dictionary 담기

In [ ]:
for number in numbers:

    json_root,_,json_list = next(os.walk(os.path.join(json_dir_name+number)))
    tiff_root,_,tiff_list = next(os.walk(os.path.join(tiff_dir_name+number)))    
    json_list.sort()
    tiff_list.sort()

    pair_dict = {}
    for index, tiff_file in enumerate(tiff_list):
        index += 1
        for json_file in json_list:
            if json_file.find(tiff_file[:-5]) == -1 : continue
            pair_dict[tiff_file] = [json_file,"C"+number+"_"+str(index).zfill(5)]
            
    all_dict.append(pair_dict)


1번 폴더를 제외한 나머지 폴더 3900개 맞아야 함

In [ ]:
# 3900 check!!
for i in range(len(all_dict)):
    print(f"folder 0{i+1} count : {len(all_dict[i])}")

In [ ]:
# json 으로 만들기.
with open(os.path.expanduser(save_dir+"/pair.json"), 'w', encoding='utf-8') as file:
    # json.dump(all_dict, file)
    pass

### json 안에 내용 변경

In [114]:
error_list = {}

for index, number in enumerate(numbers):
    
    json_root,_,json_list = next(os.walk(os.path.join(json_dir_name+number)))
    tiff_root,_,tiff_list = next(os.walk(os.path.join(tiff_dir_name+number)))    
    json_list.sort() 
    tiff_list.sort()
    
    if len(tiff_list) == 0: continue
    
    for tiff_file in tiff_list:
        json_file,id = all_dict[index].get(tiff_file)
        
        # json 불러와서 새로운 json format 만들기
        with open(os.path.expanduser(json_root+"/"+json_file), 'r', encoding='utf-8') as f:
            img_shape = cv2.imread(tiff_root+"/"+tiff_file)
            height, width,_ = img_shape.shape
    
            data = json.load(f)
            
            json_data = {}
            json_data['images'] = {'id':id,"width":width,"height":height,"file_name":id+".tiff"}
            json_data['labels'] = []
            try:
                for box_info in data['labels']:
                    label = box_info['label']
                    
                    box_info = box_info['boundingPoly']['vertices']
                    
                    xmin = box_info[0]['x']
                    xmax = box_info[2]['x']
                    ymin = box_info[0]['y']
                    ymax = box_info[2]['y']
                    
                    coordinate = [{"xmin":xmin,"ymin":ymin,"xmax":xmax,"ymax":ymax}]
                    json_data['labels'].append({"label": label, "boundingPloy":{"type":"RECT", "vertices":coordinate}})
            except KeyError:
                error_list[id] = [number, tiff_file, json_file]

        # 새로운 json format 저장하기
        with open(os.path.expanduser(json_root+"/modify/"+id+".json"), 'w', encoding='utf-8') as f:
            json.dump(json_data,f)

### 에러 파일들 확인

In [115]:
error_list
with open(os.path.expanduser("./korad_boxes/error_list.json"), 'w', encoding='utf-8') as f:
    json.dump(error_list,f)

### json & tiff 이름 변경

In [ ]:
for index, number in enumerate(numbers):
    
    json_root,_,json_list = next(os.walk(os.path.join(json_dir_name+number)))
    tiff_root,_,tiff_list = next(os.walk(os.path.join(tiff_dir_name+number)))    
    json_list.sort() 
    tiff_list.sort()
    
    if len(tiff_list) == 0: continue
    
    for tiff_file in tiff_list:
        json_file,id = all_dict[index].get(tiff_file)
        print(f"tiff_file : {tiff_file}")
        print(f"json_file : {json_file}")
        print(f"id : {id}")
        # os.rename(json_root+"/"+tiff_file,json_root+"/"+id+".json")
        # os.rename(tiff_root+"/"+json_file,tiff_root+"/"+id+".tiff")

### Visualize

In [ ]:
for number in os.listdir(tiff_dir_name):
    if number != "07":continue
    json_root,_,json_list = next(os.walk(os.path.join(json_dir_name+number)))
    tiff_root,_,tiff_list = next(os.walk(os.path.join(tiff_dir_name+number)))    
    json_list.sort()
    tiff_list.sort()

    for file in tiff_list:
        json_file = pair_dict.get(file)
        
        with open(json_root+"/"+json_file, "r") as box_j :
            cjson = json.load(box_j)

        img = cv2.imread(os.path.join(tiff_root+"/"+file))
        
        for box_info in cjson['labels']:
            cls = box_info['label']

            color_idx = classes.index(cls)
            color = colors[color_idx]
            box_info['boundingPoly']['vertices']

            xmin = box_info['boundingPoly']['vertices'][0]['x']
            xmax = box_info['boundingPoly']['vertices'][2]['x']
            ymin = box_info['boundingPoly']['vertices'][0]['y']
            ymax = box_info['boundingPoly']['vertices'][2]['y']
            img = cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 3)

        cv2.imshow("image",img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        # cv2.imwrite(os.path.join(save_dir, file.replace('.tiff', '.png')), img)
        break